In [1]:
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import StandardScaler
import pandas as pd
import numpy as np


In [2]:
data_df = pd.read_csv('columbus_accidents_regions 2.csv')

In [3]:
display(data_df.describe())

,Unnamed: 0,Severity,Start_Lat,Start_Lng,End_Lat,End_Lng,Distance(mi),Temperature(F),Wind_Chill(F),Humidity(%),Pressure(in),Visibility(mi),Wind_Speed(mph),Precipitation(in),Centroid_Lat,Centroid_Lng
count,2.221600e+04,22216.000000,22216.000000,22216.000000,8142.000000,8142.000000,22216.000000,22159.000000,16194.000000,22058.000000,22169.000000,22142.000000,21018.000000,13988.000000,22216.000000,22216.000000
mean,2.937866e+06,2.450126,39.807078,-83.691153,39.816264,-84.847632,0.570461,51.816142,43.649846,71.960831,29.475774,8.550788,8.730988,0.013672,39.807166,-83.691575
std,2.254165e+06,0.595986,1.758665,4.395942,2.508214,6.862262,1.414307,20.472849,24.807579,20.003013,0.708494,2.678965,5.088667,0.065392,1.758717,4.396167
min,1.000000e+01,1.000000,29.663366,-109.424154,29.691592,-109.671261,0.000000,-20.000000,-41.000000,8.000000,20.060000,0.000000,0.000000,0.000000,29.667046,-109.421361
25%,9.953922e+05,2.000000,39.943413,-83.036194,39.941820,-83.095610,0.000000,36.000000,26.000000,57.000000,29.100000,8.000000,5.000000,0.000000,39.943896,-83.040012
50%,2.474712e+06,2.000000,39.975233,-82.991562,39.976735,-83.000197,0.010000,52.000000,41.000000,75.000000,29.440000,10.000000,8.000000,0.000000,39.977000,-82.990403
75%,4.354020e+06,3.000000,40.036819,-82.934802,40.059882,-82.938752,0.531000,69.000000,64.000000,89.000000,30.020000,10.000000,12.000000,0.000000,40.038969,-82.934428
max,7.727027e+06,4.000000,48.822446,-74.653181,48.836445,-74.653181,40.476000,99.000000,99.000000,100.000000,30.760000,19.000000,37.000000,2.270000,48.812377,-74.640387


In [4]:
categorical_columns = data_df.select_dtypes(include=['object', 'category']).columns

In [5]:
display(categorical_columns)

Index(['ID', 'Source', 'Start_Time', 'End_Time', 'Description', 'Street',
       'City', 'County', 'State', 'Zipcode', 'Country', 'Timezone',
       'Airport_Code', 'Weather_Timestamp', 'Wind_Direction',
       'Weather_Condition', 'Sunrise_Sunset', 'Civil_Twilight',
       'Nautical_Twilight', 'Astronomical_Twilight', 'Region', 'Neighbor_1',
       'Neighbor_2', 'Neighbor_3', 'Neighbor_4', 'Neighbor_5', 'Neighbor_6',
       'Neighbor_7'],
      dtype='object')

In [13]:
columns_to_remove = ['Precipitation(in)', 'Wind_Chill(F)', 'End_Lat', 'End_Lng','Airport_Code']
data_df.drop(columns=list(columns_to_remove), inplace=True)

In [14]:
columns_to_fill = [
    'Wind_Speed(mph)', 
    'Temperature(F)', 
    'Humidity(%)',
    'Pressure(in)',
    'Visibility(mi)'
]

# Replace missing values with the mean for each column in the list
for column in columns_to_fill:
    mean_value = data_df[column].mean()
    data_df[column] = data_df[column].fillna(mean_value)

In [15]:
categorical_columns_to_fill = [
    'Weather_Timestamp', 
    'Wind_Direction', 
    'Weather_Condition', 
    'Sunrise_Sunset',
    'Civil_Twilight',
    'Nautical_Twilight',
    'Astronomical_Twilight',
    'Street'
]
# Replace missing values with the mode for each categorical column in the list
for column in categorical_columns_to_fill:
    mode_value = data_df[column].mode()[0]  # mode() returns a Series, get the first value
    data_df[column] = data_df[column].fillna(mode_value)

In [16]:
# Normalize numerical data
categorical_features = data_df.select_dtypes(exclude=[np.number]).columns.tolist()
numeric_features = data_df.select_dtypes(include=[np.number]).columns.tolist()


# Encode categorical features using oneHotEncoder, and standardize numerical features using StandardScaler
preprocessor = ColumnTransformer(
    transformers=[
        ('num', StandardScaler(), numeric_features),
        ('cat', OneHotEncoder(), categorical_features)
    ])



In [22]:
data_df_transformed = preprocessor.fit_transform(data_df)
display(data_df_transformed)


<22216x100151 sparse matrix of type '<class 'numpy.float64'>'
	with 1155111 stored elements in Compressed Sparse Row format>

In [23]:

data_df.to_csv('cleaned_data.csv', index=False)